# Batch Reactor Example
## Ignition delay computation

In this example we will illustrate how to setup and use a constant volume batch reactor. This reactor will then be used to compute the ignition delay of a gas at any temperature and pressure

The reactor (system) is simply an insulated box.

In [1]:
# Write Python source code in this cell

## Cantera Simulation Procedure

In all Cantera simulations, there are three main steps:

1. Create the phase object from an input file
2. Set the initial/boundary conditions
3. Run the simulation

In the case of a batch reactor, Cantera includes several built-in reactor models to calculate the energy and species equations subject to either constant volume or constant pressure conditions.

### Define the gas

First, we have to define the phase.

In this example we will choose n-heptane as the gas. For a representative kinetic model, we use the 160 species [mechanism](https://combustion.llnl.gov/archived-mechanisms/alkanes/heptane-reduced-mechanism) by [Seier et al. 2000, Proc. Comb. Inst](http://dx.doi.org/10.1016/S0082-0784&#40;00&#41;80610-4). 

In [2]:
# Write Python source code in this cell

### Define reactor conditions: temperature, pressure, fuel, stoichiometry

Next, we need to define initial conditions. We will choose a temperature of 1000 K, pressure of 1 atm, and equivalence ratio of 1.0. Remember to set the composition before the temperature and pressure.

In [3]:
# Write Python source code in this cell

### Run the simulation

The third step in the process is usually the most complicated. Cantera provides 4 homogeneous reactor classes:

| | Any Equation Of State | Ideal Gas Equation Of State |
|-|-|-|
| Constant Volume | `Reactor` | `IdealGasReactor` |
| Constant Pressure | `ConstPressureReactor` | `IdealGasConstPressureReactor` |

In this example, the fuel is an ideal gas and we want to model constant volume combustion, so we choose the `IdealGasReactor`.

In [4]:
# Write Python source code in this cell

Cantera supports arbitrary _networks_ of reactors that can be connected together by flow control devices. The network of reactors has to be inserted into a `ReactorNetwork` to handle the integration. All of the `Reactor`s added to a `ReactorNetwork` are integrated simultaneously! In this case, we have only the one `reactor`.

In [5]:
# Write Python source code in this cell

Last, we'd like a data structure to store the data from the time integration. Cantera has a useful data structure called the `SolutionArray` which is an array of `Solution` objects. `SolutionArray`s can also store "extra" data, such as the integration time.

In [6]:
# Write Python source code in this cell

Now all the problem setup is done and we have to perform the integration. This is done by one of 3 methods:

1. `step()`: Take one timestep, with the size determined by the CVODES integrator
2. `advance(time)`: Take many adaptively-sized timesteps until the `time` is reached
3. `advance_to_steady_state()`: Take many adaptively-sized timesteps until the system reaches steady state

We will use `step()` here to record data at every time step so that we can have a good resolution to calculate the ignition delay. Integration proceeds until the reactor network time is greater than an estimated ignition delay time.

In [7]:
# Write Python source code in this cell

Now we can use the integration data to determine the ignition delay. We will use the `"oh"` species to compute the ignition delay at the maximum mass fraction of this species. Using round parentheses with the `SolutionArray` returns data about the species that has been passed. From this, we can access the mass fraction with the `Y` attribute and then find the _index_ of the maximum of the mass fraction using `argmax()`. The index is then used to find the ignition delay time.

In [8]:
# Write Python source code in this cell

## Plot the result

### Figure illustrating the definition of ignition delay

In [9]:
# Write Python source code in this cell

## Illustration : NTC behavior

A common benchmark for a reaction mechanism is its ability to reproduce the **N**egative **T**emperature **C**oefficient behavior. Intuitively, as the temperature of an explosive mixture increases, it should ignite faster. But, under certain conditions, we observe the opposite. This is referred to as NTC behavior. Reproducing experimentally observed NTC behavior is thus an important test for any mechanism. We will do this now by computing and visualizing the ignition delay for a wide range of temperatures

### Define the temperatures for which we will run the simulations

Typical NTC behavior for n-heptane occurs from ~600-800 K, so we need to do an integration of the problem at temperatures in a wide range around those temperatures. We can use some functions from NumPy to make it easy to generate the temperatures we want.

In [10]:
# Write Python source code in this cell

As before, we will define some estimated ignition delay times to limit the integration time. These are chosen empirically and must be updated for different fuels or different conditions.

In [11]:
# Write Python source code in this cell

And last, we will create a `SolutionArray` to store the ignition delay results. This `SolutionArray` will only store the final result of calculating the ignition delay, not the time history of each simulation. In this case, we know that we want to have as many rows as the `T` array in our `SolutionArray`, so we can initialize it that way. Then we can set the initial conditions for the entire array of `Solution`s!

In [12]:
# Write Python source code in this cell

Now, we can loop through these states and use them to set the state of the `gas` object, which we can use in the `IdealGasReactor` and do the integration again.

This time, we'll only store the information we care about from the simulation, which are the time steps and the history of the reference species. We will use regular Python `list`s to store this information, since they are quite efficient when appending data rapidly.

Then we will compute the ignition delay from this data and proceed to the next state!

In [13]:
# Write Python source code in this cell

### Figure: ignition delay ($\tau$) vs. the inverse of temperature ($\frac{1000}{T}$). 

Last, we will create an Arrhenius plot of the ignition delays.

In [14]:
# Write Python source code in this cell

## Challenge question: How does EGR impact ignition delay?
Most modern internal combustion engines use exhaust gas recirculation (EGR) as a form of emissions control to reduce the formation of NO<sub>x</sub>. EGR works by recirculating some of the expanded and cooled combustion products back into the fuel/air charge in the cylinder, reducing cylinder temperatures and subsequently reducing the NO<sub>x</sub> emissions. 

For this challenge, you are tasked to evaluate the effect of increasing EGR dilution on the ignition delay for the n-heptane mixture studied earlier. Be sure to carefully consider how to properly model the exhaust products that you will introduce into the intake charge.